In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC



In [9]:
satellite = pd.read_csv("Satellite.csv",sep=';')

In [11]:
satellite.columns

Index(['x.1', 'x.2', 'x.3', 'x.4', 'x.5', 'x.6', 'x.7', 'x.8', 'x.9', 'x.10',
       'x.11', 'x.12', 'x.13', 'x.14', 'x.15', 'x.16', 'x.17', 'x.18', 'x.19',
       'x.20', 'x.21', 'x.22', 'x.23', 'x.24', 'x.25', 'x.26', 'x.27', 'x.28',
       'x.29', 'x.30', 'x.31', 'x.32', 'x.33', 'x.34', 'x.35', 'x.36',
       'classes'],
      dtype='object')

In [13]:
y = satellite['classes']
X = satellite.drop('classes',axis = 1)

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

## Linear -> SVM(default ovr)

In [17]:
svm = SVC(kernel='linear', probability = True,random_state = 24)

In [19]:
svm.fit(X_train,y_train)
y_pred = svm.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8581046090108752


## with scaling

In [23]:
svm = SVC(kernel='linear', probability = True,random_state = 24,decision_function_shape = 'ovo')
scaler_mm = MinMaxScaler()
pipe = Pipeline([('SCL',scaler_mm),('SVM',svm)])
pipe.fit(X_train,y_train)


Pipeline(steps=[('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(decision_function_shape='ovo', kernel='linear',
                     probability=True, random_state=24))])

In [25]:
y_pred = pipe.predict(X_test)

In [27]:
print(accuracy_score(y_test,y_pred))

0.865872604867944


In [31]:
y_pred_prob = pipe.predict_proba(X_test)
print(log_loss(y_test,y_pred_prob))

0.3571064276620879


## Grid SearchCV

In [35]:
svm = SVC(kernel='linear', probability = True,random_state = 24)
scaler_mm = MinMaxScaler()
pipe = Pipeline([('SCL',scaler_mm),('SVM',svm)])
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'SVM__C':np.linspace(0.001,5,3),
         'SVM__decision_function_shape':['ovo','ovr']}
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'neg_log_loss',verbose = 3)
gcv.fit(X,y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END SVM__C=0.001, SVM__decision_function_shape=ovo;, score=-0.487 total time=   9.7s
[CV 2/5] END SVM__C=0.001, SVM__decision_function_shape=ovo;, score=-0.492 total time=   9.6s
[CV 3/5] END SVM__C=0.001, SVM__decision_function_shape=ovo;, score=-0.526 total time=   9.7s
[CV 4/5] END SVM__C=0.001, SVM__decision_function_shape=ovo;, score=-0.497 total time=   9.5s
[CV 5/5] END SVM__C=0.001, SVM__decision_function_shape=ovo;, score=-0.472 total time=   8.8s
[CV 1/5] END SVM__C=0.001, SVM__decision_function_shape=ovr;, score=-0.487 total time=   9.5s
[CV 2/5] END SVM__C=0.001, SVM__decision_function_shape=ovr;, score=-0.492 total time=   9.4s
[CV 3/5] END SVM__C=0.001, SVM__decision_function_shape=ovr;, score=-0.526 total time=   9.9s
[CV 4/5] END SVM__C=0.001, SVM__decision_function_shape=ovr;, score=-0.497 total time=  10.0s
[CV 5/5] END SVM__C=0.001, SVM__decision_function_shape=ovr;, score=-0.472 total time=   9.9

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'SVM__decision_function_shape': ['ovo', 'ovr']},
             scoring='neg_log_loss', verbose=3)

In [38]:
gcv.best_estimator_

Pipeline(steps=[('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=5.0, decision_function_shape='ovo', kernel='linear',
                     probability=True, random_state=24))])

In [40]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0, 'SVM__decision_function_shape': 'ovo'}
-0.33206246904927117


In [56]:
svm = SVC(kernel='rbf', probability = True,random_state = 24)
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)  
pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])
params = {'SVM__C':np.linspace(0.001,5,3),'SVM__gamma': np.linspace(0.001,5,3) }

In [ ]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'neg_log_loss',verbose = 3)
gcv.fit(X,y)

In [52]:
gcv.best_estimator_

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000295B8019310>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000295B97EB260>)],
                                   verbose_feature_names_out=False)),
                ('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=0.001, gamma=0.001, probability=True, random_state=24))])

In [54]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 0.001, 'SVM__gamma': 0.001}
nan


## Radial Kernel

In [72]:
svm = SVC(kernel='rbf', probability = True,random_state = 24)
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)


In [74]:
pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])
params = {'SVM__C':np.linspace(0.001,5,3),'SVM__gamma': np.linspace(0.001,5,3),
         'SVM__decision_function_shape':['ovo','ovr']}

In [76]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'neg_log_loss',verbose = 3)
gcv.fit(X,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=0.001;, score=-0.850 total time=  19.4s
[CV 2/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=0.001;, score=-0.821 total time=  18.9s
[CV 3/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=0.001;, score=-0.791 total time=  19.4s
[CV 4/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=0.001;, score=-0.762 total time=  19.3s
[CV 5/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=0.001;, score=-0.877 total time=  18.7s
[CV 1/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=2.5004999999999997;, score=-0.636 total time=  18.2s
[CV 2/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=2.5004999999999997;, score=-0.550 total time=  19.0s
[CV 3/5] END SVM__C=0.001, SVM__decision_function_shape=ovo, SVM__gamma=2.5004999999999997;, score=-0.546 total time=  18.7s
[CV 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000295BD03BEC0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._colu...ke_column_selector object at 0x00000295BD4CEF00>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.0000e-03, 2.5005e+00, 5.0000e+00]),
                         'SVM__decision_function_shape': ['ovo', 'ovr'],
                         'SVM__gamma': array([1.0000e-03, 2.5005e+00, 5.0000e+00])},
             scoring='neg_log_loss', verbose=3)

In [78]:
gcv.best_estimator_

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000295BD4CC350>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000295BD4CF3E0>)],
                                   verbose_feature_names_out=False)),
                ('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=5.0, decision_function_shape='ovo', gamma=5.0,
                     probability=True, random_state=24))])

In [80]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0, 'SVM__decision_function_shape': 'ovo', 'SVM__gamma': 5.0}
-0.2140689944828622
